In [ ]:
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")
NUM_CLASSES = 

/opt/miniconda3/envs/LING413/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 820719.61 examples/s]


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = imdb_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:08<00:00, 5947.56 examples/s]


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=NUM_CLASSES
)

In [ ]:
import torch.nn as nn
from transformers import BertModel

class RegisterClassifier(nn.Module):
    def __init__(self, n_classes):
        super(RegisterClassifier, self).__init__()
        # Load the pretrained BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        # Dropout layer for regularization
        self.drop = nn.Dropout(p=0.3)
        
        # Fully-connected layer for classification
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        # Pass inputs through BERT
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # Use the pooled output of the token for classification
        pooled_output = outputs.pooler_output
        
        # Apply dropout and the final classification layer
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
import torch
# Assuming 'model' is your trained PyTorch model
# and 'output_dir' is the directory to save the model
output_dir = "model_weights"
model_save_path = f"{output_dir}/final_model_weights.pth"
torch.save(model.state_dict(), model_save_path)

In [ ]:
from transformers import AutoModelForSequenceClassification

# Instantiate the same model architecture
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=NUM_CLASSES
)

# Load the saved weights
model.load_state_dict(torch.load(model_save_path))

# Set the model to evaluation mode
model.eval()

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

# Assume 'model' is loaded and in eval mode, and 'test_loader' is ready
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_preds =
all_labels =

test_loader = 

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        preds = torch.argmax(logits, dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate the classification report
# 'target_names' should be a list of your class names
report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)

# Convert the report to a pandas DataFrame and save as a CSV file [64]
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(f"{output_dir}/classification_report.csv")

print("Classification Report:")
print(report_df)

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

# Assume 'model' is loaded and in eval mode, and 'test_loader' is ready
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_preds =
all_labels =

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        preds = torch.argmax(logits, dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate the classification report
# 'target_names' should be a list of your class names
report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)

# Convert the report to a pandas DataFrame and save as a CSV file [64]
report_df = pd.DataFrame(report).transpose()
report_df.to_csv(f"{output_dir}/classification_report.csv")

print("Classification Report:")
print(report_df)

In [ ]:
import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super(MLPClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        # text shape: (batch_size, seq_len)
        embedded = self.embedding(text)
        # Average the embeddings of all words in a sentence
        pooled = embedded.mean(dim=1)
        # Pass through linear layers
        hidden = self.relu(self.fc1(pooled))
        return self.fc2(hidden)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, num_filters, filter_sizes):
        super(CNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, 
                      out_channels=num_filters, 
                      kernel_size=(fs, embed_dim)) 
            for fs in filter_sizes
        ])
        
        self.fc = nn.Linear(len(filter_sizes) * num_filters, num_class)
        self.dropout = nn.Dropout(0.5)

    def forward(self, text):
        # text shape: (batch_size, seq_len)
        embedded = self.embedding(text)
        # embedded shape: (batch_size, seq_len, embed_dim)
        
        embedded = embedded.unsqueeze(1)
        # embedded shape: (batch_size, 1, seq_len, embed_dim)
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        # conved[i] shape: (batch_size, num_filters, seq_len - filter_sizes[i] + 1)
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # pooled[i] shape: (batch_size, num_filters)
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        # cat shape: (batch_size, num_filters * len(filter_sizes))
        
        return self.fc(cat)